<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#第８章　化合物リスト提出・総括" data-toc-modified-id="第８章　化合物リスト提出・総括-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>第８章　化合物リスト提出・総括</a></span></li><li><span><a href="#アッセイ候補提出ファイルの作成" data-toc-modified-id="アッセイ候補提出ファイルの作成-2"><span class="toc-item-num">2&nbsp;&nbsp;</span><b><u>アッセイ候補提出ファイルの作成</u></b></a></span></li><li><span><a href="#LBVS-or-SBVS" data-toc-modified-id="LBVS-or-SBVS-3"><span class="toc-item-num">3&nbsp;&nbsp;</span><b><u>LBVS or SBVS</u></b></a></span><ul class="toc-item"><li><span><a href="#①-Structure-Based-Virtual-Screening（SBVS）" data-toc-modified-id="①-Structure-Based-Virtual-Screening（SBVS）-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>① Structure-Based Virtual Screening（SBVS）</a></span></li><li><span><a href="#②-Ligand-Based-Virtual-Screening（LBVS）" data-toc-modified-id="②-Ligand-Based-Virtual-Screening（LBVS）-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>② Ligand-Based Virtual Screening（LBVS）</a></span></li></ul></li><li><span><a href="#総括" data-toc-modified-id="総括-4"><span class="toc-item-num">4&nbsp;&nbsp;</span><b><u>総括</u></b></a></span></li></ul></div>

# 第８章　化合物リスト提出・総括

# <b><u>アッセイ候補提出ファイルの作成</b></u>

新しいスコア関数 RF-Score VSによる再スコアリングと結果の取り出しが完了しました。AutoDock Vinaのスコアリングとそれぞれ違いがあって面白いので、各々Top 5を取り出して合わせて10化合物とし、アッセイ候補としたいと思います。（ダイバーシティー！）

提出フォーマットはタブ区切り、ということなのでcsvモジュールを使ってファイルを作ります。まずはリストを作成。

```python
top_10_list = []

# RFScoreVSを基準に5個取り出し
RF_sup = Chem.SDMolSupplier('./RF-ScoreVS_Top10.sdf')

for i in range(5):
    temp_list =[]
    mol = RF_sup[i]

    rank_num = i + 1
    compound_id = mol.GetProp('original_id')
    score = mol.GetProp('RFScoreVS_v2')
    
    temp_list = [rank_num, compound_id,score]
    top_10_list.append(temp_list)

# AutoDock Vinaを基準に5個取り出し
ADVina_sup = Chem.SDMolSupplier('./SBVS_Top10.sdf')

for i in range(5):
    temp_list =[]
    mol = ADVina_sup[i]

    rank_num = i + 6
    compound_id = mol.GetProp('original_id')
    score = mol.GetProp('vina_affinity')
    
    temp_list = [rank_num, compound_id,score]
    top_10_list.append(temp_list)
```

10化合物についてIDとスコアの組み合わせのリストのリストが作成できたので、タブ区切りのファイルを書き出します。

```python
import csv
with open('./top10_magattaca.txt', 'w') as f:
    writer = csv.writer(f, delimiter='\t')
    writer.writerows(top_10_list)
```

念のため読み込んで確認

```python
with open('./top10_magattaca.txt') as f:
    print(f.read())
"""
1	Z393761442	6.438919
2	Z243276918	6.404471
3	Z200290200	6.386205
4	Z368242106	6.385228
5	Z278994950	6.207442
6	Z237879862	-10.9
7	Z19455181	-10.9
8	Z872606870	-10.8
9	Z2788907736	-10.4
10	PB90021090	-10.4
"""
```

無事目的のファイルが作成できていそうです。果たしてこの中に活性のあるものはあるでしょうか？？？  

同様に、500化合物のリストも同じファイル形式としておきます。スコアとしてはLBVSを行なったときの類似性評価のTanimoto係数によるスコアを記載しておきます。　（色々こねくり回して494個と500個に少し足りませんがご容赦いただけると信じて）

```python
Top500_sup = Chem.SDMolSupplier('./SCR_compounds_SimScore.sdf')
top_500_list = []
rank_num = 0

for mol in Top500_sup:
    rank_num +=1
    compound_id = mol.GetProp('original_id')
    score = mol.GetProp('Similarity_Score')
    temp_list = [rank_num, compound_id,score]
    top_500_list.append(temp_list)
```

完了！　これを提出フォームに登録すればOK！

# <b><u>LBVS or SBVS</u></b>

提出も完了し、余裕ができたところで全くもって今更ですが**Virtual Screening**の２つの手法、**LBVS**と**SBVS**についてその特徴をまとめておきたいと思います。

## ① Structure-Based Virtual Screening（SBVS）

SBVSでは**標的タンパク質の構造情報**に基づき、結合するリガンドを探索します。

具体的には標的タンパク質の結合サイトにリガンドをおき、

* 形状からみたはまり具合の良さを評価する（**ドッキングシミュレーション**）
* タンパク質 - リガンド間相互作用の強さを評価する（**スコアリング**）

ことにより、タンパク質とリガンドの親和性を見積もります。

結合サイトの情報をもとにするため、

* 既知リガンド情報がなくてもスクリーニングの実施が可能、また、
* 既知の活性化合物と類似性の低い**新規な骨格**を発見できる可能性がある

といった長所があります。一方で、扱う情報量（原子の数）が多いので**計算コストが高い**のが短所だそうです。

## ② Ligand-Based Virtual Screening（LBVS）

LBVSでは**既知活性化合物の構造**にもとづき、**化学的類似性**をもとにスクリーニングを行います。

長所は

* SBVSよりも計算コストが低く
* 標的タンパク質の構造が未知でも実施可能

なことです。

一方で、類似性に基づく以上**新規な骨格を見つけにくい**という短所があるそうです。

　また、LBVSの別のアプローチとして**ファーマコフォアモデル**を利用する手法もあります。ファーマコフォアモデルは複数の活性化合物の情報から、活性に必須な構造を抜き出して作成するモデルで、このモデルとの類似性から化合物探索を行うそうです。

それぞれの特徴と創薬レイドバトル開始時点でのPD-1/PD-L1における状況をまとめると以下のようになるでしょうか？

||長所| 短所| 創薬レイド| 
|:---:|:---|:---|:---|
| SBVS |・新規骨格の探索に有効<br>・タンパク質とリガンドの結合エネルギー評価可能 |・標的タンパク質の立体構造が必要<br>・計算コストが高い|・PD-1/PD-L1それぞれの結晶構造あり<br>・低分子との共結晶構造あり |
| LBVS |・タンパク質の構造が未知でも良い<br>・計算コストが低い| ・既知活性化合物の情報が必要<br>・新規骨格の探索に不向き |・特許、文献に低分子化合物の情報 |

# <b><u>総括</b></u>

最後にこれまで行なってきた処理手順をまとめます。

* 1. 前処理（脱塩、構造標準化）  
* 2. 指標による絞り込み（主にRule of 5の適用）
* 3. 部分構造による絞り込み（オルト置換ビフェニル構造）
* 4. ファーマコフォアスクリーニング（ファーマコフォアを変えて２回実施）
        ➡︎ 化合物 Top 500 (厳密には494個)
* 5. LBVS（既知活性化合物に対するフィンガープリントベースの類似性評価による順位づけ）
        ➡︎ 化合物 Top 50を次工程へ 
* 6. SBVS (AutoDock Vina + RF-Score VS)
　　　　 ➡︎ 化合物 Top 10をアッセイ希望に選択

以上の６ステップです。書いてしまうとこれだけ、という感じもありますが、各工程でびっくりするくらいに行き詰まったので予想以上に時間がかかってしまいました。

今回行なった処理の手順を踏まえて、先にまとめたLBVS、SBVSそれぞれの特徴を考えると、「Step 5. LBVS」で**類似性を基準**にスクリーニングをかけたのちに、「Step 6. SBVS」を行なっているため、SBVSの**新規な骨格を発見**できるという長所を活かせていない処理手順となってしまっていました。一方で、SBVSの短所である**計算コストの高さ**という面を考慮すると、私の貧弱な計算環境ではたくさんの化合物のドッキング計算の実施は現実的には困難です。（50個でも数時間かかりました）

ですので、LBVSで絞りこんでからドッキング計算を行うという手順にもある程度のメリットはあったのではないでしょうか？（Twitterで拝見した手順を真似ただけなのに偉そうなことを言ってすみません・・・）

展望も戦略もないままはじめて右往左往していましたが、行き詰まるたびにTwitterで解決方法や、方針、文献諸々を一からご教示いただき、なんともありがたい気持ちでいっぱいです。この場を借りて皆様にお礼申し上げます。

あとは、選んだ化合物の中に活性を持つものがあることを祈って！果報は寝て待ちましょう。  
